In [2]:
import os
import openai
from dotenv import load_dotenv
import sys
sys.path.append('../..')

# Obtén la ruta absoluta del archivo openai.env
env_path = '/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/openai.env'

load_dotenv(dotenv_path=env_path)

api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [6]:
import pandas as pd
import openai
import time
import re

def obtener_nombre_ecommerce(texto):
    matches_simple = re.findall(r"'([^']*)'", texto)  # Encuentra textos en comillas simples
    matches_double = re.findall(r'"([^"]*)"', texto)  # Encuentra textos en comillas dobles
    matches = matches_simple + matches_double  # Combina los resultados
    if matches:
        # Retorna el último match
        result = matches[-1]
    else:
        # Si no hay comillas, retorna el texto original
        result = texto

    # Define los prefijos a eliminar
    prefixes = ["Marca - ", "Nombre del producto - ", "Cantidad - ", "Color - ", "Otras características relevantes - "]

    # Elimina cada prefijo si está presente
    for prefix in prefixes:
        if result.startswith(prefix):
            result = result[len(prefix):]  # El prefijo tiene un número de caracteres igual a su longitud

    # Si el resultado contiene la línea de cabecera, la elimina
    header = "Marca - Nombre del producto - Cantidad - Color - Otras características relevantes:"
    if header in result:
        result = result.replace(header, "").strip()  # Elimina la cabecera y los espacios en blanco al principio y al final

    return result


def procesar_proveedor(proveedor):
    path = f'/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/data/proveedor/{proveedor}/listos/{proveedor}.xlsx'

    # Lee el archivo xlsx y selecciona las columnas requeridas
    df = pd.read_excel(path, engine='openpyxl')
    df = df[['Identificador de URL', 'canal_Nombre', 'df2_Nombre','canal_Marca']]

    # Elimina las filas con valores NaN en las columnas 'Identificador de URL' y 'canal_Nombre'
    df = df.dropna(subset=['Identificador de URL', 'canal_Nombre','df2_Nombre'])

    # Crear una copia del DataFrame original
    df_original = df.copy()

    # Eliminar la columna 'Identificador de URL', 'canal_Marca' y 'canal_Nombre' del DataFrame de trabajo
    df = df.drop(['canal_Nombre', 'Identificador de URL'], axis=1)

    # Lee el archivo de ejemplos
    df_ejemplos = pd.read_excel('ejemplos.xlsx')
    ejemplos = df_ejemplos.to_dict('records')
    # Formatea los ejemplos para la entrada de la API de OpenAI
    ejemplos_formateados = [f"{ejemplo['original']} -> '{ejemplo['corregido']}'" for ejemplo in ejemplos]
    ejemplos_string = "\n".join(ejemplos_formateados)

    df['nombre_ecommerce'] = ''

    print(df.head())
    contador = 0

    for i, row in df.iterrows():
        marca = "" if pd.isna(row['canal_Marca']) else f". Marca del producto: '{row['canal_Marca']}'"
        messages = [
            {"role": "system", "content": "Tienes la tarea de generar nombres atractivos para productos para el comercio electrónico, siguiendo el formato: 'Marca - Nombre del producto - Cantidad - Color - Otras características relevantes'."},
            {"role": "user", "content": f"""Aquí están algunos ejemplos de nombres de productos renovados: 
        {ejemplos_string}
        Ahora, genera un nombre atractivo para el siguiente producto. Nombre del producto original: '{row['df2_Nombre']}'{marca}."""}
        ]

    # ... el resto de tu código ...


        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=100,
            temperature=0.5
        )

        # Asigna la respuesta del modelo a la columna correspondiente
        df.at[i, 'nombre_ecommerce'] = response['choices'][0]['message']['content'].strip()

        # Imprime la información de la iteración actual
        print(f"Iteración {i}")
        print(f"Nombre original del producto: {row['df2_Nombre']}")
        print(f"Nombre nuevo del producto: {response['choices'][0]['message']['content'].strip()}")
        print("------------------------")

        contador += 1
        if contador % 100 == 0:
            print("Tomando un receso de 30 segundos...")
            time.sleep(30)

    # Combina df_original y df en base al índice
    df_final = df_original.merge(df, left_index=True, right_index=True)

    # Selecciona solo las columnas 'Identificador de URL' y 'nombre_ecommerce'
    df_final = df_final[['Identificador de URL', 'nombre_ecommerce']]

    df_final['nombre_ecommerce'] = df_final['nombre_ecommerce'].apply(obtener_nombre_ecommerce)

    # Renombra la columna 'nombre_ecommerce' a 'Nombre'
    df_final = df_final.rename(columns={'nombre_ecommerce': 'Nombre'})

    # Guarda el DataFrame a un nuevo archivo .xlsx
    df_final.to_excel(f'{proveedor}_results.xlsx', index=False)


In [7]:
procesar_proveedor('furey')


                                 df2_Nombre  canal_Marca nombre_ecommerce
0         ACEITE DE RICINO TABLADA  x 40 cc      Tablada                 
1  CH CHUP CORAZON C/ESTUCHE ANAT.+ 0M ROSA  Chapoteando                 
2                      CH ESTUCHE N.20 ROSA  Chapoteando                 
3                   CH ESTUCHE N.10 CELESTE  Chapoteando                 
4                      CH ESTUCHE N.10 ROSA  Chapoteando                 
Iteración 0
Nombre original del producto: ACEITE DE RICINO TABLADA  x 40 cc
Nombre nuevo del producto: Tablada - Aceite de Ricino - 40 cc - Natural y Nutritivo
------------------------
Iteración 1
Nombre original del producto: CH CHUP CORAZON C/ESTUCHE ANAT.+ 0M ROSA
Nombre nuevo del producto: Chapoteando - Chupete corazón con estuche tetina anatómico + 0 meses - Rosa
------------------------
Iteración 2
Nombre original del producto: CH ESTUCHE N.20 ROSA
Nombre nuevo del producto: Chapoteando - Estuche 20 - Rosa Encantador
------------------------
It

KeyboardInterrupt: 